In [22]:
import pandas as pd
import os,glob
from datetime import datetime, timedelta

In [ ]:
# 将城市的多个指标合并到一个文件中
zp_path = "D:\文档\坚果云\区域经济\数字经济\数字经济指数\数据\人工智能招聘数据\福建省人工智能招聘量周统计2022-2024.xlsx" # 招聘量
zl_path = "D:\文档\坚果云\区域经济\数字经济\数字经济指数\数据\专利\福建省专利申请量周统计2022-2024.xlsx" # 专利申请量
baidu_index_path = "D:\文档\坚果云\区域经济\数字经济\数字经济指数\数据\百度指数_数字经济\数字经济指标" # 百度指数

In [ ]:
# 各城市
citys = ["福州",
        "莆田",
        "三明",
        "龙岩",
        "厦门",
        "泉州",
        "漳州",
        "宁德",
        "南平"]

In [25]:
zp_df = pd.read_excel(zp_path)
zl_df = pd.read_excel(zl_path)

In [26]:
zp_df

,福州,厦门,南平,泉州,宁德,三明,龙岩,莆田,漳州,week
0,4,3,0,1,0,0,0,0,0,2021-52
1,32,26,1,6,8,0,0,5,3,2022-01
2,18,18,0,6,3,0,1,1,3,2022-02
3,23,21,0,1,2,0,0,1,0,2022-03
4,10,20,0,5,2,0,0,0,0,2022-04
...,...,...,...,...,...,...,...,...,...,...
153,10,2,2,0,0,0,0,0,0,2024-49
154,10,8,0,0,0,0,0,0,0,2024-50
155,6,2,0,4,0,0,0,0,0,2024-51
156,2,1,0,0,0,0,0,0,0,2024-52


In [27]:
zl_df

,福州市,厦门市,泉州市,宁德市,漳州市,莆田市,南平市,龙岩市,三明市,week
0,0,3,0,0,0,0,0,0,0,2021-52
1,99,132,60,42,15,7,6,21,8,2022-01
2,236,190,83,12,19,27,7,11,10,2022-02
3,197,237,136,16,23,16,5,31,12,2022-03
4,298,345,125,28,46,18,9,36,14,2022-04
...,...,...,...,...,...,...,...,...,...,...
153,170,169,63,17,11,6,10,5,2,2024-49
154,189,201,65,18,25,6,9,11,5,2024-50
155,187,220,64,8,9,8,8,8,9,2024-51
156,214,262,60,9,28,4,3,16,7,2024-52


In [28]:
def get_resylt_df():
    start_date = datetime(2022, 1, 1)
    end_date = datetime(2024, 12, 31)
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    all_dates = [d.strftime('%Y-%m-%d') for d in date_range]

    # 3. 获取所有城市列表
    columns = ['专利','招聘','云计算与AI','产业数字化','数字化治理','数据价值化']

    # 4. 创建完整的时间-城市矩阵
    result = pd.DataFrame(index=all_dates, columns=columns).fillna(0)
    result['日期'] = result.index
    result['日期'] = pd.to_datetime(result['日期'])

    result['week'] = result['日期'].dt.strftime('%G-%V')  # 格式如"2022-52"
      # 6. 生成每周日期范围说明列
    result['week_date_range'] = result.groupby('week')['日期'].transform(
        lambda x: f"{x.min().strftime('%Y-%m-%d')} 至 {x.max().strftime('%Y-%m-%d')}"
    )
    
    # 7. 按周聚合（保留日期范围信息）
    weekly_result = result.groupby('week').agg({
        **{col: 'sum' for col in columns},  # 指标列求和
        'week_date_range': 'first'          # 取每周末日期范围
    }).reset_index()

    return pd.DataFrame(weekly_result)

In [29]:
pd.read_excel(os.path.join(baidu_index_path,"福州"+"_产业数字化.xlsx"))

,亚马逊,京东,唯品会,工业互联网,快手,抖音,拼多多,智慧农业,智慧医疗,智慧园区,智慧教育,智能交通,智能制造,智能工厂,朴朴,淘宝,美团,饿了么,每周总和,week
0,700,2689,610,114,1348,3557,2082,182,0,119,0,0,59,0,539,3280,1322,771,17372,2021-52
1,2961,10066,2091,462,4130,12435,6774,729,412,348,346,172,704,0,1970,13236,3891,2199,62926,2022-01
2,2814,10878,2209,417,3940,12365,5700,673,479,348,237,171,464,0,1913,13543,3734,1971,61856,2022-02
3,2660,11160,2384,641,4698,12266,5730,559,117,357,230,57,344,0,2017,13055,4050,2260,62585,2022-03
4,2462,11315,1810,527,5068,11982,5429,593,285,352,230,116,114,0,2133,11118,4654,2343,60531,2022-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,1946,9388,1877,498,6873,30664,6434,647,57,427,474,243,470,299,3424,13118,6334,2637,85810,2024-49
154,1803,9194,2025,452,6342,30162,6040,950,59,508,238,238,460,171,3506,12680,5696,2442,82966,2024-50
155,1729,8684,1964,187,5631,29340,6122,928,57,351,347,323,345,57,3583,11948,5870,2451,79917,2024-51
156,1745,8667,1956,665,6220,28987,5975,722,176,347,285,0,409,57,3580,11818,5997,2570,80176,2024-52


In [30]:
zp_df["福州"]

0       4
1      32
2      18
3      23
4      10
       ..
153    10
154    10
155     6
156     2
157     0
Name: 福州, Length: 158, dtype: int64

In [31]:
for city in citys:
    result_df = get_resylt_df()
    result_df['招聘'] = zp_df[city].set_axis(result_df.index)
    result_df['专利'] = zl_df[city+"市"].set_axis(result_df.index)
    # 百度指数
    result_df['云计算与AI'] = pd.read_excel(os.path.join(baidu_index_path,city+"_数字产业化.xlsx"))['每周总和'].set_axis(result_df.index)
    result_df['产业数字化'] = pd.read_excel(os.path.join(baidu_index_path,city+"_产业数字化.xlsx"))['每周总和'].set_axis(result_df.index)
    result_df['数字化治理'] = pd.read_excel(os.path.join(baidu_index_path,city+"_数字化治理.xlsx"))['每周总和'].set_axis(result_df.index)
    result_df['数据价值化'] = pd.read_excel(os.path.join(baidu_index_path,city+"_数据价值化.xlsx"))['每周总和'].set_axis(result_df.index)
    result_df.to_excel(city+"数字经济指标汇总2022-2024.xlsx",index=False)